# GLM Analysis
This notebook reproduces the GLM analysis of the paper. The output is all of the .nii volumes containing the contrasts used in the figures.

In [1]:
import os

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import pandas as pd
import statsmodels.stats.multitest
from nipy.modalities.fmri.glm import GeneralLinearModel
from scipy import signal
from scipy.ndimage import binary_dilation
from scipy.special import gamma

ModuleNotFoundError: No module named 'nipy'

## Location of the data
Change DATA_DIR to point to the data. The data should be structured as follows:

    <DATA_DIR>/sub-XX/func/sub-XX_task-salinemorphine_bold.nii
    <DATA_DIR>/sub-XX/func/sub-XX_task-salinemorphine_events.tsv
    <DATA_DIR>/derivatives/sub-XX/func/sub-XX_task-salinemorphine_space-orig_desc-brain_mask.nii
    <DATA_DIR>/derivatives/sub-XX/func/sub-XX_task-salinemorphine_confounds.txt

Where XX goes from 01 to 12.

The outputs will be saved in

    <DATA_DIR>/derivatives/all/contrasts/

In [1]:
DATA_DIR = '../../data'

In [2]:
def fmri_path(subject_id):
    return os.path.join(DATA_DIR, f'sub-{subject_id}', 'func', f'sub-{subject_id}_task-salinemorphine_bold.nii')

def events_path(subject_id):
    return os.path.join(DATA_DIR, f'sub-{subject_id}', 'func', f'sub-{subject_id}_task-salinemorphine_events.tsv')

def mask_path(subject_id):
    return os.path.join(DATA_DIR, 'derivatives', f'sub-{subject_id}', 'func', f'sub-{subject_id}_task-salinemorphine_space-orig_desc-brain_mask.nii')

def realignment_parameters_path(subject_id):
    return os.path.join(DATA_DIR, 'derivatives', f'sub-{subject_id}', 'func', f'sub-{subject_id}_task-salinemorphine_confounds.txt')

## Define functions

The next few blocks define the function used to perform the analysis. The main function is <code>fit_model</code> which does most of the work.

In [4]:
def split_clusters(image):
    """Splits an image into connected clusters of non-zero voxels"""
    
    # Find all the non-zero elements.
    voxels = set(tuple(v) for v in np.argwhere(image))

    clusters = []
    while len(voxels) > 0:
        
        cluster = [voxels.pop()]
        clusters.append(cluster)
        
        while True:
            mask = np.zeros(image.shape, dtype=bool)
            for voxel in cluster:
                mask[voxel] = True
            region = set(tuple(v) for v in np.argwhere(binary_dilation(mask)))
            new_voxels = voxels & region
            if len(new_voxels) == 0:
                break
            else:
                cluster.extend(new_voxels)
                voxels -= new_voxels
        
    return clusters

In [5]:
def window_events(events, window_size: int):
    """Split events by window of a fixed size.
    
    Because the events are very long (600 seconds), this function splits then into shorter events. This allows us
    to treat each window as its own event.

    """
    event_types = list(set(events['trial_type']))

    new_events = []    
    for i, event_type in enumerate(event_types):
        current_events = events.loc[events['trial_type'] == event_type]
        for onset, duration in zip(current_events['onset'], current_events['duration']):
            n_windows = int(duration // window_size)
            for i in range(n_windows):
                new_events.append({'onset': onset + i * window_size, 'duration': window_size, 'trial_type': event_type + f'-w{i:02d}'})
            remainder = duration - n_windows * window_size
            if remainder > 0:
                new_events.append({'onset': onset + n_windows * window_size, 'duration': remainder, 'trial_type': event_type + f'-w{i:02d}'})
            
    return pd.DataFrame(new_events)

In [6]:
def hrf(t):
    """Rat specific HRF"""
    b = 2.0
    p1 = 7.4
    p2 = 8.9
    V = 1.5
    return np.exp(-b * t) * ((b ** p1) / gamma(p1) * t ** (p1 - 1) - (b ** p2) / (V * gamma(p2)) * t **(p2 - 1))

def assemble_design_matrix(times, events, confounds=None, intercept=True):
    """Asssembles the design matrix for the GLM."""
    
    rows = []
    event_types = np.unique(events['trial_type'])
    for event_type in event_types:
        current_events = events.loc[events['trial_type'] == event_type]
        row = np.zeros(len(times))
        for onset, duration in zip(current_events['onset'], current_events['duration']):
            row[np.logical_and(times >= onset, times <= onset + duration)] = 1
            
        row = np.convolve(row, hrf(np.arange(len(times)) * (times[1] - times[0])), 'full')[:len(times)]
        rows.append(row)
        
    if intercept:
        rows.append(np.ones((len(times))))
        
    if confounds is not None:
        rows.extend(confounds)
    
    return np.array(rows).T, list(event_types)

In [7]:
def fit_model(subject_ids, window_size=30.0, repetition_time=2.0):
    """Fits the GLM model for all rats with the provided IDs.

    This function can be used to fit the model to a single individual by providing a single ID, or
    on many individuals simultaneously by providing many ids.

    """
    
    all_data = []
    all_design_matrix = []
    
    for subject_id in subject_ids:
    
        # Load data.
        fmri_nii = nib.load(fmri_path(subject_id))
        fmri = fmri_nii.get_fdata()

        events = pd.read_csv(events_path(subject_id), sep='\t')
        events = window_events(events, window_size)

        brain_mask_nii = nib.load(mask_path(subject_id))
        brain_mask = brain_mask_nii.get_fdata().astype(bool)

        # Preprocess
        fmri_preprocessed = fmri.reshape((np.prod(fmri_nii.shape[:3]), fmri_nii.shape[3]), order='F')
        fmri_preprocessed = fmri_preprocessed[brain_mask.reshape((np.prod(fmri_nii.shape[:3]),), order='F'), :]
        fmri_preprocessed = fmri_preprocessed
        fmri_preprocessed -= np.mean(fmri_preprocessed, axis=1, keepdims=True)

        shape = fmri_nii.shape[:3] + (fmri_nii.shape[-1],)
        times = np.arange(shape[3]) * repetition_time

        # Filter.
        b, a = signal.butter(8, (0.01, 0.1), btype='bandpass', fs=1.0/repetition_time)
        fmri_filtered = signal.filtfilt(b, a, fmri_preprocessed)
        
        # Confounds.
        realignment_parameters = np.loadtxt(realignment_parameters_path(subject_id))
        confounds = np.hstack((realignment_parameters, np.mean(fmri_filtered, axis=0)[:, None]))

        # Design matrix. Ignore the background and skip the first 5 volumes.
        data = fmri_filtered[:, 5:].T
        design_matrix, event_types = assemble_design_matrix(times[5:], events, confounds=confounds[5:, :].T)
        
        all_data.append(data)
        all_design_matrix.append(design_matrix)

    design_matrix = np.concatenate(all_design_matrix, axis=0)
    data = np.concatenate(all_data, axis=0)

    model = GeneralLinearModel(design_matrix)
    model.fit(data)
    
    return model, event_types


## Fit the model
Here we fit the model for all rats simultaneously.

In [18]:
subject_ids = [f'{i + 1:02d}' for i in range(12)]
window_size = 30.0
min_cluster_size = 16
model, event_types = fit_model(subject_ids, window_size=window_size, repetition_time=2.0)

In [20]:
windows = [f'{i:02d}' for i in range(int(600 // window_size))]

# Keep the number of significant voxels for each window. Used to plot the overall results after.
nonzeros_mor_sal = []
nonzeros_sal_mor = []
nonzeros_mor = []
nonzeros_sal = []
nonzeros_nmor = []
nonzeros_nsal = []

for w in windows:
    
    b = model.get_beta()
    
    ## Morphine - saline
    cval = np.zeros((model.X.shape[1],))
    cval[event_types.index(f'saline-w{w}')] = -1
    cval[event_types.index(f'morphine-w{w}')] = 1
    p_values = model.contrast(cval).p_value()
    rejected, corrected_p_values = statsmodels.stats.multitest.fdrcorrection(p_values, alpha=0.05)

    # Mask based on cluster size.
    mask = np.zeros((np.prod(fmri_nii.shape[:3]),), dtype=np.uint8)
    mask[brain_mask.ravel('F')] = rejected
    mask = mask.reshape(fmri_nii.shape[:3], order='F')
    clusters = split_clusters(mask)
    clusters = [c for c in clusters if len(c) >= min_cluster_size]
    mask = np.zeros(fmri_nii.shape[:3], dtype=np.uint8)
    for cluster in clusters:
        for voxel in cluster:
            mask[tuple(voxel)] = 1
    nonzeros_mor_sal.append(np.count_nonzero(mask))

    p_data = np.ones((np.prod(fmri_nii.shape[:3]),), dtype=np.float64)
    p_data[brain_mask.ravel('F')] = corrected_p_values
    p_data = p_data.reshape(fmri_nii.shape[:3], order='F')
    p_data = p_data * mask
    directory = os.path.join(DATA_DIR, 'derivatives', 'all', 'contrasts')
    os.makedirs(directory, exist_ok=True)
    image = nib.Nifti1Image(p_data, fmri_nii.affine)
    nib.save(image, os.path.join(directory, f'p_mor-sal-w{w}.nii'))

    p_data = np.zeros((np.prod(fmri_nii.shape[:3]),), dtype=np.uint8)
    p_data[brain_mask.ravel('F')] = rejected
    p_data = p_data.reshape(fmri_nii.shape[:3], order='F')
    p_data = p_data * mask
    directory = os.path.join(DATA_DIR, 'derivatives', 'all', 'contrasts')
    os.makedirs(directory, exist_ok=True)
    image = nib.Nifti1Image(p_data, fmri_nii.affine)
    nib.save(image, os.path.join(directory, f'r_mor-sal-w{w}.nii'))
    
    ## Saline - Morphine
    cval = np.zeros((model.X.shape[1],))
    cval[event_types.index(f'saline-w{w}')] = 1
    cval[event_types.index(f'morphine-w{w}')] = -1
    p_values = model.contrast(cval).p_value()
    rejected, corrected_p_values = statsmodels.stats.multitest.fdrcorrection(p_values, alpha=0.05)

    # Mask based on cluster size.
    mask = np.zeros((np.prod(fmri_nii.shape[:3]),), dtype=np.uint8)
    mask[brain_mask.ravel('F')] = rejected
    mask = mask.reshape(fmri_nii.shape[:3], order='F')
    clusters = split_clusters(mask)
    clusters = [c for c in clusters if len(c) >= min_cluster_size]
    mask = np.zeros(fmri_nii.shape[:3], dtype=np.uint8)
    for cluster in clusters:
        for voxel in cluster:
            mask[tuple(voxel)] = 1
    nonzeros_sal_mor.append(np.count_nonzero(mask))

    p_data = np.ones((np.prod(fmri_nii.shape[:3]),), dtype=np.float64)
    p_data[brain_mask.ravel('F')] = corrected_p_values
    p_data = p_data.reshape(fmri_nii.shape[:3], order='F')
    p_data = p_data * mask
    directory = os.path.join(DATA_DIR, 'derivatives', 'all', 'contrasts')
    os.makedirs(directory, exist_ok=True)
    image = nib.Nifti1Image(p_data, fmri_nii.affine)
    nib.save(image, os.path.join(directory, f'p_sal-mor-w{w}.nii'))

    p_data = np.zeros((np.prod(fmri_nii.shape[:3]),), dtype=np.uint8)
    p_data[brain_mask.ravel('F')] = rejected
    p_data = p_data.reshape(fmri_nii.shape[:3], order='F')
    p_data = p_data * mask
    directory = os.path.join(DATA_DIR, 'derivatives', 'all', 'contrasts')
    os.makedirs(directory, exist_ok=True)
    image = nib.Nifti1Image(p_data, fmri_nii.affine)
    nib.save(image, os.path.join(directory, f'r_sal-mor-w{w}.nii'))
    
    ## Saline
    cval = np.zeros((model.X.shape[1],))
    cval[event_types.index(f'saline-w{w}')] = 1
    p_values = model.contrast(cval).p_value()
    rejected, corrected_p_values = statsmodels.stats.multitest.fdrcorrection(p_values, alpha=0.05)

    # Mask based on cluster size.
    mask = np.zeros((np.prod(fmri_nii.shape[:3]),), dtype=np.uint8)
    mask[brain_mask.ravel('F')] = rejected
    mask = mask.reshape(fmri_nii.shape[:3], order='F')
    clusters = split_clusters(mask)
    clusters = [c for c in clusters if len(c) >= min_cluster_size]
    mask = np.zeros(fmri_nii.shape[:3], dtype=np.uint8)
    for cluster in clusters:
        for voxel in cluster:
            mask[tuple(voxel)] = 1
    nonzeros_sal.append(np.count_nonzero(mask))

    p_data = np.ones((np.prod(fmri_nii.shape[:3]),), dtype=np.float64)
    p_data[brain_mask.ravel('F')] = corrected_p_values
    p_data = p_data.reshape(fmri_nii.shape[:3], order='F')
    p_data = p_data * mask
    directory = os.path.join(DATA_DIR, 'derivatives', 'all', 'contrasts')
    os.makedirs(directory, exist_ok=True)
    image = nib.Nifti1Image(p_data, fmri_nii.affine)
    nib.save(image, os.path.join(directory, f'p_sal-w{w}.nii'))

    p_data = np.zeros((np.prod(fmri_nii.shape[:3]),), dtype=np.uint8)
    p_data[brain_mask.ravel('F')] = rejected
    p_data = p_data.reshape(fmri_nii.shape[:3], order='F')
    p_data = p_data * mask
    directory = os.path.join(DATA_DIR, 'derivatives', 'all', 'contrasts')
    os.makedirs(directory, exist_ok=True)
    image = nib.Nifti1Image(p_data, fmri_nii.affine)
    nib.save(image, os.path.join(directory, f'r_sal-w{w}.nii'))
    
    ## Morphine
    cval = np.zeros((model.X.shape[1],))
    cval[event_types.index(f'morphine-w{w}')] = 1
    p_values = model.contrast(cval).p_value()
    rejected, corrected_p_values = statsmodels.stats.multitest.fdrcorrection(p_values, alpha=0.05)

    # Mask based on cluster size.
    mask = np.zeros((np.prod(fmri_nii.shape[:3]),), dtype=np.uint8)
    mask[brain_mask.ravel('F')] = rejected
    mask = mask.reshape(fmri_nii.shape[:3], order='F')
    clusters = split_clusters(mask)
    clusters = [c for c in clusters if len(c) >= min_cluster_size]
    mask = np.zeros(fmri_nii.shape[:3], dtype=np.uint8)
    for cluster in clusters:
        for voxel in cluster:
            mask[tuple(voxel)] = 1
    nonzeros_mor.append(np.count_nonzero(mask))
            
    p_data = np.ones((np.prod(fmri_nii.shape[:3]),), dtype=np.float64)
    p_data[brain_mask.ravel('F')] = corrected_p_values
    p_data = p_data.reshape(fmri_nii.shape[:3], order='F')
    p_data = p_data * mask
    directory = os.path.join(DATA_DIR, 'derivatives', 'all', 'contrasts')
    os.makedirs(directory, exist_ok=True)
    image = nib.Nifti1Image(p_data, fmri_nii.affine)
    nib.save(image, os.path.join(directory, f'p_mor-w{w}.nii'))

    p_data = np.zeros((np.prod(fmri_nii.shape[:3]),), dtype=np.uint8)
    p_data[brain_mask.ravel('F')] = rejected
    p_data = p_data.reshape(fmri_nii.shape[:3], order='F')
    p_data = p_data * mask
    directory = os.path.join(DATA_DIR, 'derivatives', 'all', 'contrasts')
    os.makedirs(directory, exist_ok=True)
    image = nib.Nifti1Image(p_data, fmri_nii.affine)
    nib.save(image, os.path.join(directory, f'r_mor-w{w}.nii'))
    
    ## Negative Saline
    cval = np.zeros((model.X.shape[1],))
    cval[event_types.index(f'saline-w{w}')] = -1
    p_values = model.contrast(cval).p_value()
    rejected, corrected_p_values = statsmodels.stats.multitest.fdrcorrection(p_values, alpha=0.05)

    # Mask based on cluster size.
    mask = np.zeros((np.prod(fmri_nii.shape[:3]),), dtype=np.uint8)
    mask[brain_mask.ravel('F')] = rejected
    mask = mask.reshape(fmri_nii.shape[:3], order='F')
    clusters = split_clusters(mask)
    clusters = [c for c in clusters if len(c) >= min_cluster_size]
    mask = np.zeros(fmri_nii.shape[:3], dtype=np.uint8)
    for cluster in clusters:
        for voxel in cluster:
            mask[tuple(voxel)] = 1
    nonzeros_nsal.append(np.count_nonzero(mask))

    p_data = np.ones((np.prod(fmri_nii.shape[:3]),), dtype=np.float64)
    p_data[brain_mask.ravel('F')] = corrected_p_values
    p_data = p_data.reshape(fmri_nii.shape[:3], order='F')
    p_data = p_data * mask
    directory = os.path.join(DATA_DIR, 'derivatives', 'all', 'contrasts')
    os.makedirs(directory, exist_ok=True)
    image = nib.Nifti1Image(p_data, fmri_nii.affine)
    nib.save(image, os.path.join(directory, f'p_nsal-w{w}.nii'))

    p_data = np.zeros((np.prod(fmri_nii.shape[:3]),), dtype=np.uint8)
    p_data[brain_mask.ravel('F')] = rejected
    p_data = p_data.reshape(fmri_nii.shape[:3], order='F')
    p_data = p_data * mask
    directory = os.path.join(DATA_DIR, 'derivatives', 'all', 'contrasts')
    os.makedirs(directory, exist_ok=True)
    image = nib.Nifti1Image(p_data, fmri_nii.affine)
    nib.save(image, os.path.join(directory, f'r_nsal-w{w}.nii'))
    
    ## Negative Morphine
    cval = np.zeros((model.X.shape[1],))
    cval[event_types.index(f'morphine-w{w}')] = -1
    p_values = model.contrast(cval).p_value()
    rejected, corrected_p_values = statsmodels.stats.multitest.fdrcorrection(p_values, alpha=0.05)

    # Mask based on cluster size.
    mask = np.zeros((np.prod(fmri_nii.shape[:3]),), dtype=np.uint8)
    mask[brain_mask.ravel('F')] = rejected
    mask = mask.reshape(fmri_nii.shape[:3], order='F')
    clusters = split_clusters(mask)
    clusters = [c for c in clusters if len(c) >= min_cluster_size]
    mask = np.zeros(fmri_nii.shape[:3], dtype=np.uint8)
    for cluster in clusters:
        for voxel in cluster:
            mask[tuple(voxel)] = 1
    nonzeros_nmor.append(np.count_nonzero(mask))
            
    p_data = np.ones((np.prod(fmri_nii.shape[:3]),), dtype=np.float64)
    p_data[brain_mask.ravel('F')] = corrected_p_values
    p_data = p_data.reshape(fmri_nii.shape[:3], order='F')
    p_data = p_data * mask
    directory = os.path.join(DATA_DIR, 'derivatives', 'all', 'contrasts')
    os.makedirs(directory, exist_ok=True)
    image = nib.Nifti1Image(p_data, fmri_nii.affine)
    nib.save(image, os.path.join(directory, f'p_nmor-w{w}.nii'))

    p_data = np.zeros((np.prod(fmri_nii.shape[:3]),), dtype=np.uint8)
    p_data[brain_mask.ravel('F')] = rejected
    p_data = p_data.reshape(fmri_nii.shape[:3], order='F')
    p_data = p_data * mask
    directory = os.path.join(DATA_DIR, 'derivatives', 'all', 'contrasts')
    os.makedirs(directory, exist_ok=True)
    image = nib.Nifti1Image(p_data, fmri_nii.affine)
    nib.save(image, os.path.join(directory, f'r_nmor-w{w}.nii'))
    
plt.figure(dpi=150)
plt.plot(nonzeros_mor_sal)
plt.plot(nonzeros_sal_mor)
plt.xlabel('Window number')
plt.ylabel('Number of significant voxels')
plt.ylim([0, 3000])
plt.legend(['Morphine - Saline', 'Saline - Morphine'])
plt.savefig(os.path.join(directory, f'n_significant_voxels_mor-sal+sal-mor.png'))
plt.close()

plt.figure(dpi=150)
plt.plot(nonzeros_sal)
plt.plot(nonzeros_nsal)
plt.xlabel('Window number')
plt.ylabel('Number of significant voxels')
plt.ylim([0, 3000])
plt.legend(['Positive Saline', 'Negative Saline'])
plt.savefig(os.path.join(directory, f'n_significant_voxels_sal+nsal.png'))
plt.close()

plt.figure(dpi=150)
plt.plot(nonzeros_mor)
plt.plot(nonzeros_nmor)
plt.xlabel('Window number')
plt.ylabel('Number of significant voxels')
plt.ylim([0, 3000])
plt.legend(['Positive Morphine', 'Negative Morphine'])
plt.savefig(os.path.join(directory, f'n_significant_voxels_mor+nmor.png'))
plt.close()

0.0 80.09583750829049
float64
0.0 164.14465686099737
0.0 148.70853682203744
0.0 95.86896128192191
0.0 0.0
float64
0.0 0.0
0.0 84.70995925336192
0.0 69.40345336589516
0.0 0.0
float64
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
float64
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
float64
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
float64
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
float64
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
float64
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
float64
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
float64
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
float64
0.0 0.0
0.0 0.0
0.0 0.0
0.0 106.85908391112227
float64
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
float64
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
float64
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
float64
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
float64
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
float64
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
float64
0.0 0.0
0.0 0.0
0.0 0.0
0.0 105.93482558129978
float64
0.0 95.47149246218483
0.0 70.1820748366497
0.0 99.57852417651986
0.0 105.36951607972699
float64
0.0 75.63040802029565
0.0 80.928576888052
0.0 121.86